In [ ]:
from db import databaseconnection
from ga import networkedgeneticalgorithm as nga

In [ ]:
dbconn = databaseconnection.DatabaseConnection('db/datastore.db')

In [ ]:
dbconn.whatSessions()

In [ ]:
data = dbconn.loadSession('2018-02-17 10:00:20')

In [ ]:
print(data['metrics'])
print(data['genealogy']['tree'])
print(data['genealogy']['history'])
print(len(data['individuals']))
print(data['individuals'][0])

In [ ]:
dbconn.close()